# Mixed SM 2021 

<table><tr>
<td><img src="https://raw.githubusercontent.com/elastufka/presentations/master/images/ZU_logo1.png" width="200"></td>
 <td><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSOgmNJ347YwD-sfSxjATPgVkPQh6XknjCv4Q&usqp=CAU" width='200'></td></tr>
    </table>
    
 **Some (incomplete) statistics, now with more information content**
 

## About the Plots

They are fully interactive! Just in case you have not encountered this before... it should be pretty easy to figure out when you hover your mouse in random places and it tells you what you can do.

I would not advise looking at this on a mobile phone.

## About the Data
**Skip if you're impatient**

- Statistics taken with UltiAnalytics app using GoPro footage. I couldn't always see the endzones...

- Limitations of UltiAnalytics: All turns from our team are either drops or throwaways. So if a player got D'd it's counted as one of those (usually depending on how close to the camera it happened - if it was closer chances are higher I counted it as a drop because I could see what was going on)

- Timing data was useless here since I was often pausing or rewinding the video feed to try and figure out who was who 

- Data is anonymized since this is public (but probably hard to find) 

- Anonymized data and code available [here]()

In [1]:
#py37
import pandas as pd 
import plotly
from plotly import graph_objects as go
import plotly.express as px
from PIL import ImageColor
from ultistats_utils import *
from visualization_tools import *

In [2]:
#slideshow stuff
from IPython.display import Javascript
from plotly.offline import get_plotlyjs
Javascript(get_plotlyjs())
slidew=850
slideh=600
set_pride_template()

In [3]:
pcolors=pride_colors_plotly()
bcolors=pride_colors_plotly(flag='bi')
logo_layout=global_layout()

## Load & Prep Data

In [219]:
df=pd.read_csv('FucZH-stats.csv')

In [220]:
df.drop(columns=list(df.keys())[20:],inplace=True)

In [222]:
df['Passer_matchup']=['W' if p in wline else 'M' if p in mline else 'U' if p== 'Anonymous' else p for p in df.Passer]
df['Receiver_matchup']=['W' if p in wline else 'M' if p in mline else 'U' if p== 'Anonymous' else p for p in df.Receiver]
df['Defender_matchup']=['W' if p in wline else 'M' if p in mline else 'U' if p== 'Anonymous' else p for p in df.Defender]

In [224]:
break_type,hockey_assist,assist,scorer,gng,gngm,gngn,opp_ta=[],[],[],[],[],[],[],[]
turns,touches=[],[]
W4_M3=[]
M4_W3=[]

for idx,row in df.iterrows():
    #print(row,row['Line'])
    turns.append(is_turn(row))
    touches.append(sort_touches(row))
    assist.append(is_assist(row))
    scorer.append(goal_scorer(row))
    seven_on=[row[k] for k in row.keys() if 'Player' in k]
    #print(seven_on)
    if gender_ratio(seven_on,wline,mline) == 1:
        W4_M3.append(1)
        M4_W3.append(0)
    else:
        W4_M3.append(0)
        M4_W3.append(1)
    #opponent throwaway?
    if row.Action=='Throwaway' and type(row.Passer)==float: #nan
        opp_ta.append(1)
    else:
        opp_ta.append(0)
    try:
        isgng,gm,gn=give_and_go(df,idx,row)
        if type(gn)==list:
            gn.sort()
            gn=tuple(gn)
        gng.append(isgng)
        gngm.append(gm)
        gngn.append(gn)
        hockey_assist.append(is_hockey_assist(df,idx,row))
    except (KeyError,IndexError) as e:
        hockey_assist.append(None)
        gng.append(None)
        gngm.append(None)
        gngn.append(None)
        pass
    if idx >0:
        break_type.append(is_break(row['Line'],row['Action'],row['Receiver']))
        #print(row['Line'],row['Action'],row['Receiver'],break_type)
    else:
        break_type.append(None)

In [225]:
#anonymize - might need to run this at the end just in case
for k,v in anony_dict.items():
    df.replace(k,v,inplace=True)

In [226]:
df['W4_M3']=W4_M3
df['M4_W3']=M4_W3
df['Offense_line']=[1 if l == 'O' else 0 for l in df.Line]

In [227]:
game_dict={'Fly High Lausanne':1,'Wombats Willisau':2,'Wizards Geneva':3,'FSL':4,'FSL (Quarters)':5,'Fly High (Semis)':6,'Crazy Dogs':7}

In [228]:
df['Game_Number']=[game_dict[row.Opponent] for _,row in df.iterrows()]

In [229]:
df['Break']=break_type
df['Hockey_assist']=hockey_assist
df['Scorer']=scorer
df['Assist']=assist
df['Turn']=turns
df['Touches']=touches
df['give-and-go']=gng
df['give-and-go_matchup']=gngm
df['give-and-go_pairs']=gngn
df['opponent_throwaway']=opp_ta

In [230]:
df['Pass_matchup']=df.Passer_matchup+df.Receiver_matchup
df['PointID']=[o+f"{op:02d}"+f"{tp:02d}" for o,op,tp in zip(df.Opponent,df['Our Score - End of Point'],df['Their Score - End of Point'])]

In [237]:
#dummies for 'Action' colump
adf=pd.get_dummies(df.Action)
#bdf=pd.get_dummies(df.Break)
#pmdf0=pd.get_dummies(df.Pass_matchup)
#pmdf0.rename(columns={'MW':'Pass_matchup_MW','WM':'Pass_matchup_WM','MM':'Pass_matchup_MM','WW':'Pass_matchup_WW','MU':'Pass_matchup_MU','UM':'Pass_matchup_UM','WU':'Pass_matchup_WU','UW':'Pass_matchup_UW'},inplace=True)
pmdf=pd.get_dummies(df.Touches)
pmdf.rename(columns={'U':'Touch_U','W':'Touch_W','M':'Touch_M'},inplace=True)
# rmdf=pd.get_dummies(df.Receiver_matchup)
# rmdf.rename(columns={'U':'Receiver_matchup_U','W':'Receiver_matchup_W','M':'Receiver_matchup_M'},inplace=True)
# dmdf=pd.get_dummies(df.Defender_matchup)
# dmdf.rename(columns={'U':'Defender_matchup_U','W':'Defender_matchup_W','M':'Defender_matchup_M'},inplace=True)
df=df.merge(adf,left_index=True,right_index=True)
#df=df.merge(bdf,left_index=True,right_index=True)
#df=df.merge(pmdf0,left_index=True,right_index=True)
df=df.merge(pmdf,left_index=True,right_index=True)
#df=df.merge(rmdf,left_index=True,right_index=True)
#df=df.merge(dmdf,left_index=True,right_index=True)
df.head(10)

,Date/Time,Tournamemnt,Opponent,Point Elapsed Seconds,Line,Our Score - End of Point,Their Score - End of Point,Event Type,Action,Passer,...,EndOfThirdQuarter,GameOver,Goal,Halftime,Pull,PullOb,Throwaway,Touch_M,Touch_U,Touch_W
0,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,130,D,0,1,Defense,Pull,NaN,...,0,0,0,0,1,0,0,0,0,0
1,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,130,D,0,1,Defense,D,NaN,...,0,0,0,0,0,0,0,1,0,0
2,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,130,D,0,1,Offense,Catch,FS,...,0,0,0,0,0,0,0,1,0,0
3,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,130,D,0,1,Offense,Catch,CS,...,0,0,0,0,0,0,0,1,0,0
4,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,130,D,0,1,Offense,Drop,SG,...,0,0,0,0,0,0,0,1,0,0
5,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,130,D,0,1,Defense,Goal,NaN,...,0,0,1,0,0,0,0,0,0,0
6,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,163,O,1,1,Offense,Catch,SH,...,0,0,0,0,0,0,0,1,0,0
7,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,163,O,1,1,Offense,Catch,GA,...,0,0,0,0,0,0,0,1,0,0
8,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,163,O,1,1,Offense,Catch,FS,...,0,0,0,0,0,0,0,1,0,0
9,2021-09-20 13:34,Mixed SM 2021,Crazy Dogs,163,O,1,1,Offense,Catch,GA,...,0,0,0,0,0,0,0,1,0,0


In [238]:
df.keys()

Index(['Date/Time', 'Tournamemnt', 'Opponent', 'Point Elapsed Seconds', 'Line',
       'Our Score - End of Point', 'Their Score - End of Point', 'Event Type',
       'Action', 'Passer', 'Receiver', 'Defender', 'Hang Time (secs)',
       'Player 0', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5',
       'Player 6', 'Passer_matchup', 'Receiver_matchup', 'Defender_matchup',
       'W4_M3', 'M4_W3', 'Offense_line', 'Game_Number', 'Break',
       'Hockey_assist', 'Scorer', 'Assist', 'Turn', 'Touches', 'give-and-go',
       'give-and-go_matchup', 'give-and-go_pairs', 'opponent_throwaway',
       'Pass_matchup', 'PointID', 'Catch', 'D', 'Drop', 'EndOfFirstQuarter',
       'EndOfThirdQuarter', 'GameOver', 'Goal', 'Halftime', 'Pull', 'PullOb',
       'Throwaway', 'Touch_M', 'Touch_U', 'Touch_W'],
      dtype='object')

In [239]:
df.to_csv('FucZH_SM_anonymized.csv')

### group by point

In [40]:
gdf=df.groupby('PointID')

## Plots - overall

## Game progression

Definitions:
- Hold = Start & Score on Offense
- Break = Start & Score on Defense

## Round 1

In [94]:
ordered_games=['Fly High Lausanne','Wombats Willisau','Wizards Geneva','FSL','FSL (Quarters)','Fly High (Semis)','Crazy Dogs']

In [147]:
fig=game_flow_fig(df,'Fly High Lausanne',slidew,slideh)
fig.show()

In [148]:
fig=game_flow_fig(df,'Wombats Willisau',slidew,slideh)
fig.show()

In [149]:
fig=game_flow_fig(df,'Wizards Geneva',slidew,slideh)
fig.show()

In [150]:
fig=game_flow_fig(df,'FSL (Quarters)',slidew,slideh)
fig.show()

In [240]:
fig=game_flow_fig(df,'FSL',slidew,slideh)
fig.show()

## Round 2

In [152]:
fig=game_flow_fig(df,'FSL (Quarters)',slidew,slideh)
fig.show()

In [153]:
fig=game_flow_fig(df,'Fly High (Semis)',slidew,slideh)
fig.show()

In [154]:
fig=game_flow_fig(df,'Crazy Dogs',slidew,slideh)
fig.show()

### Throwing and Receiving

In [155]:
#throws per offense, throws per defense
#fig=px.pie()
odf=df.where(df.Line=='O').where(df.opponent_throwaway == 0).dropna(how='all').groupby('Game_Number')
ddf=df.where(df.Line=='D').where(df.opponent_throwaway == 0).dropna(how='all').groupby('Game_Number')
tadf=df.where(df.opponent_throwaway==1).groupby('Game_Number')

In [156]:
#defense_points=
#total_points=
offense_points=df.where(df.Offense_line==1).groupby('Game_Number').PointID.nunique()
defense_points=df.where(df.Offense_line==0).groupby('Game_Number').PointID.nunique()
total_points=offense_points+defense_points

In [248]:
fig=go.Figure()
fig.add_trace(go.Bar(x=ordered_games,y=odf.Catch.sum()/offense_points,name='Offense catches'))
fig.add_trace(go.Bar(x=ordered_games,y=ddf.Catch.sum()/defense_points,name='Defense catches'))
fig.add_trace(go.Bar(x=ordered_games,y=odf.Throwaway.sum()/offense_points,name='Offense throwaway'))
fig.add_trace(go.Bar(x=ordered_games,y=ddf.Throwaway.sum()/defense_points,name='Defense throwaway'))
fig.add_trace(go.Bar(x=ordered_games,y=odf.Drop.sum()/offense_points,name="Offense drops/D'd"))
fig.add_trace(go.Bar(x=ordered_games,y=ddf.Drop.sum()/defense_points,name="Defense drops/D'd"))
fig.add_trace(go.Bar(x=ordered_games,y=odf.D.sum()/offense_points,name='Offense Ds'))
fig.add_trace(go.Bar(x=ordered_games,y=ddf.D.sum()/defense_points,name='Defense Ds'))
fig.add_trace(go.Bar(x=ordered_games,y=tadf.Throwaway.sum()/total_points,marker_color='grey',name='Opponent Throwaway'))
fig.update_layout(title='Average Events per Point')
fig.update_layout(logo_layout,width=slidew,height=slideh)

"Offense" and "Defense" here refer to line, not if we were currently playing O or D

In [249]:
ag=df.groupby('Hockey_assist').count().sort_values('Action',ascending=False)
aa=df.groupby('Assist').count().sort_values('Action',ascending=False)
fig=go.Figure()
fig.add_trace(go.Bar(x=ag.index,y=ag.Action.values,marker_color=pcolors[5],name='Hockey Assists'))
fig.add_trace(go.Bar(x=aa.index,y=aa.Action.values,marker_color=pcolors[6],name='Assists'))
fig.update_layout(barmode='stack',title='Assists and Hockey Assists',yaxis_title='Number',xaxis_title='Player',xaxis={'categoryorder':'total descending'})
fig.update_layout(logo_layout,width=slidew,height=slideh)

In [264]:
## pass matchups by gender
#ag=df.groupby('Pass_matchup').count().sort_values('Action',ascending=False)
fig=go.Figure()
for i,op in enumerate(ordered_games):
    ag=df.where(df.Opponent==op).groupby('Pass_matchup').count().sort_values('Action',ascending=False)
    fig.add_trace(go.Bar(x=ag.index,y=ag.Action,name=op,marker_color=pcolors[7-i]))
fig.update_layout(barmode='stack',title='Pass matchups',yaxis_title='Number of Passes',xaxis_title='Gender Matchup',xaxis={'categoryorder':'total descending'})
fig.update_layout(logo_layout,width=slidew,height=slideh)

In [263]:
fig=go.Figure()
for i,op in enumerate(ordered_games):
    ag=df.where(df.Opponent==op).groupby('give-and-go_matchup').count().sort_values('Action',ascending=False)
    fig.add_trace(go.Bar(x=ag.index,y=ag.Action,name=op,marker_color=pcolors[7-i]))
fig.update_layout(barmode='stack',title='Give-and-go matchups',yaxis_title='Number of Give-and-Go Passes',xaxis_title='Gender Matchup',xaxis={'categoryorder':'total descending'})
fig.update_layout(logo_layout,width=slidew,height=slideh)

A give-and-go was defined as a single instance where the thrower became the next receiver. So a sequence that looks like A-B-A-B-A is counted not as a single give-and-go, but as 3 (2 for A-B, 1 for B-A).

The order of the person in the give-and-go-pair does not matter for counting purposes.

In [252]:
ag=df.groupby('give-and-go_pairs').count().sort_values('Action',ascending=False).head(19)
xax=[p[0]+'-'+p[1] for p in ag.index.values]
colorscale = plotly.colors.sequential.Plasma
cdict=[[i/(len(colorscale)-1),c] for i,c in enumerate(colorscale)]
ccolors=[get_continuous_color(cdict,en/ag.Action.max()) for en in ag.Action.values]
fig=go.Figure()
fig.add_trace(go.Bar(x=xax,y=ag.Action.values,marker_color=ccolors))
fig.update_layout(title='Give-and-go Pairs',yaxis_title='Number of Give-and-Go Passes',xaxis_title='Pairs')#,xaxis={'categoryorder':'total descending'})
fig.update_layout(logo_layout,width=slidew,height=slideh)

In [167]:
#gender score - touch ratio / gender ratio
maj_men,maj_women=[],[]
for op in ordered_games:
    opdf=df.where(df.Opponent==op).dropna(how='all').groupby('PointID')
    tdf=opdf.agg({'W4_M3':np.median,'Touch_M':sum,'Touch_W':sum,'Touch_U':sum})
    aa=tdf.groupby('W4_M3')[['Touch_M','Touch_W','Touch_U']].mean()
    maj_men.append(aa.iloc[0].values)
    maj_women.append(aa.iloc[1].values)

In [168]:
maj_men=np.array(maj_men)
maj_women=np.array(maj_women)
maj_men.shape,maj_men[:,0]

((7, 3),
 array([6.7       , 7.25      , 3.09090909, 2.        , 4.3       ,
        5.9       , 4.4       ]))

In [256]:
#fig=go.Figure()
#for i,g in enumerate(['M','W']):
#    for j,op in enumerate(df.Opponent.unique()[:1]):
#        fig.add_trace(go.Bar(x=[-maj_women[j][i],maj_men[j][i]],name=op,hovertext=g))
#fig.show()
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=maj_men[:,0],name='M touches (4M)',orientation='h',marker_color='green'),secondary_y=False)
fig.add_trace(go.Bar(x=maj_men[:,1],name='W touches (4M)',orientation='h',marker_color='orange'),secondary_y=False)
fig.add_vrect(x0=-7,x1=0,fillcolor='purple',opacity=.2)
fig.add_trace(go.Bar(x=-1*maj_women[:,0],name='M touches (4W)',orientation='h',marker_color='green'),secondary_y=True)
fig.add_trace(go.Bar(x=-1*maj_women[:,1],name='W touches (4W)',orientation='h',marker_color='orange'),secondary_y=True)
fig.add_trace(go.Scatter(x=maj_men[:,0]/maj_men[:,1],y=list(range(7)),mode='markers',marker_symbol='cross',marker_color='black',name='m/f touch ratio'),secondary_y=False)
fig.add_trace(go.Scatter(x=-maj_women[:,1]/maj_women[:,0],y=list(range(7)),mode='markers',marker_symbol='cross',marker_color='black',name='f/m touch ratio'),secondary_y=True)
fig.add_vline(x=4/3,line_dash='dash',line_width=.5)
fig.add_vline(x=-4/3,line_dash='dash',line_width=.5,name='4:3 ratio')
fig.update_yaxes(ticktext=[],tickvals=[])
fig.update_xaxes(ticktext=[8,6,4,2,0,2,4,6,8],tickvals=[-8,-6,-4,-2,0,2,4,6,8])
fig.update_yaxes(ticktext=ordered_games,tickvals=[0,1,2,3,4,5,6],secondary_y=True)
fig.update_layout(title='Average Touches per Point',xaxis_title='Touches per Point')
fig.update_layout(legend=dict(orientation='h',yanchor='top',y=1.12))
fig.update_layout(logo_layout,width=slidew,height=slideh)

The dashed lines indicate the 4:3 ratio
    
Shaded area is where we played 4W 3M, unshaded is 4M 3W


## Fake flight path chart

In [170]:
def marker_characteristics(ndict,v):
        return v,sum(ndict[v].values())
    
def fake_coords(msizes,min_separation_factor=2): #scale is max diff between marker sizes
    recalc=True
    niter=0
    xrand=[np.random.randint(-50,50) for i in range(len(msizes))]
    yrand=[np.random.randint(-20,20) for i in range(len(msizes))]

    while recalc:
        niter+=1
        if niter == 100:
            recalc=False
        #for this the max marker radius (size ~100) will be about 10 data units
        cpairs=[(x,y) for x,y in zip(xrand,yrand)]
        for i,pair in enumerate(cpairs):
            too_close=[is_too_close(pair,rp,np.sqrt(msizes[j])*min_separation_factor) for j,rp in enumerate(cpairs)]
            if True in too_close:
                tcidx=too_close.index(True) #first one
                xrand[tcidx]=np.random.randint(100) #assign new coords
                yrand[tcidx]=np.random.randint(100)
                break
            elif i == len(cpairs)-1: #made it!
                recalc=False
    return xrand,yrand

def is_too_close(p1,p2,factor):
    if p1 == p2:
        return False
    if cartesian_distance(p1[0],p1[1],p2[0],p2[1]) < factor:
        return True
    else:
        return False

def cartesian_distance(x1,y1,x2,y2):
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def lower_opacity(rgb_in,opacity=0.2):
    if rgb_in.startswith('#'): #it's hex
        r,g,b=ImageColor.getcolor(rgb_in, "RGB")
    else:
        r=rgb_in[rgb_in.find('(')+1:rgb_in.find(',')-1]
        g=rgb_in[rgb_in.find(',')+1:rgb_in.rfind(',')-1]
        b=rgb_in[rgb_in.rfind(',')+1:-1]
    return f"rgba({r}0,{g}0,{b}0,{opacity})" #add zeros at end to ensure float

In [192]:
ndict={}
anony_list=list(anony_dict.values())
for i,v in enumerate(anony_list):
    others=anony_list[:i]+anony_list[i+1:]+['Anonymous']
    ndict[v]={o:0 for o in others}

for idx,row in df.iterrows():
    for i,v in enumerate(anony_list):
        if row.Passer == v:
            rr=row.Receiver
            if type(rr) !=float:
                ndict[v][rr]=ndict[v][rr]+1

In [193]:
msizes=[]
for v in anony_list:
    _,msize=marker_characteristics(ndict,v)
    msizes.append(msize/2.)
np.max(msizes),msizes.index(np.max(msizes)),np.min(msizes)

(80.0, 2, 3.5)

In [194]:
fakex,fakey=fake_coords(msizes)
for i,v in enumerate(anony_list): 
    ndict[v]['coords']=(fakex[i],fakey[i])

In [175]:
colorscale = plotly.colors.sequential.Magma
cdict=[[i/(len(colorscale)-1),c] for i,c in enumerate(colorscale)]
ccolors=[get_continuous_color(cdict,np.sqrt(en)/13) for en in msizes] #change from np.max(msizes) so that max color isn't so light

In [258]:
fig = go.Figure()

for j,v0 in enumerate(anony_list):
    for i,v in enumerate(anony_list):
        if v!=v0:
            if ndict[v0][v] !=0:
                fig.add_trace(
        go.Scattergeo(
            lon = [fakex[j], ndict[v]['coords'][0]],
            lat = [fakey[j], ndict[v]['coords'][1]],
            mode = 'lines',
            hoverinfo = 'text',
            text=f"{ndict[v0][v]} passes from {v0} to {v}",
            line = dict(color = lower_opacity(ccolors[j],opacity=.4),width=ndict[v0][v]),
            legendgroup=v0,showlegend=False))
                
for j,v0 in enumerate(anony_list):
    if msizes[j] > 30:
        tp="middle center"
        tc='gray'
    else:
        tp="bottom center"
        tc='black'
    fig.add_trace(go.Scattergeo(
    lon = [fakex[j]],
    lat = [fakey[j]],
    name=v0,
    legendgroup=v0,
    hoverinfo = 'text',
    hovertext = msizes[j]*2,
    text=v0,
    mode = 'markers+text',
    textposition=tp,
    textfont_color=tc,
    marker = dict(
        size = np.array(msizes[j])/1.1,
        color = ccolors[j],
        line = dict(
            width = .5,
            color = 'black')
        )
    ))

fig.update_geos(visible=False,fitbounds="locations")
fig.update_layout(logo_layout,width=slidew,height=slideh)
fig.update_layout(title='Completed Passes',height=500, margin={"r":0,"t":40,"l":0,"b":0},showlegend = True,plot_bgcolor='light gray',xaxis_visible=False)